# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Defaulting to user installation because normal site-packages is not writeable


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [3]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

In [4]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
]

In [5]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [6]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

C:\Users\lenovo\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
C:\Users\lenovo\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [7]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [8]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [9]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [10]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题']


In [11]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题


In [12]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [13]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题

<< INPUT >>
{input}

<< OUTPUT (must include ```json at the start of the

In [14]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [15]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': 'What is blackbody radiation?'}
> Finished chain.
{'input': 'What is blackbody radiation?', 'text': '\n\n黑体辐射是一种物理现象，指的是由于物体内部的热运动导致的电磁辐射。简单来说，物体在受热时会发出电磁波，这些波的频率和强度取决于物体的温度。黑体辐射的特点是它的辐射频率与物体的温度无关，而只与物体的结构和性质有关。这个概念在热力学和量子力学中都有重要的应用。'}


In [16]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们需要确定大于40的质数。40不是质数，我们从41开始检查。41是质数，但41+1=42不能被3整除。接下来是43，43+1=44也不能被3整除。然后是47，47+1=48可以被3整除。所以大于40的第一个质数是47，47+1=48可以被3整除。'}


In [17]:
router_chain.verbose = True

In [18]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是一种极端密度和强大引力的天体，它的重力场非常强大，甚至连光也无法逃脱。它的存在是由于恒星在死亡时发生了坍缩，使得它们的质量集中在极小的空间内。黑洞的存在已经被多项观测和理论证据所证实，但仍然有很多未解之谜。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [1]:
import os
 
os.environ['OPENAI_API_KEY'] = 'sk-6KpfygRF1Nm5uEW5A0756aDeC4114dA5Af7b7e518e5230Eb'
os.environ['OPENAI_BASE_URL'] = 'https://api.xiaoai.plus/v1'

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [3]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位很棒的生物学家。你擅长回答生物学问题。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

computer_template = """你是一位很棒的计算机科学家。你擅长回答计算机相关问题。
之所以如此出色，是因为你知晓计算机的历史与发展，并且了解前沿的计算机知识。

这是一个问题：
{input}"""

chinese_template = """你是一位很棒的汉语言文学家。你擅长回答汉语言文学问题。
你知晓中华文化上下五千年，在回答时可以引经据典。

这是一个问题：
{input}"""

In [4]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_template,
    },
]

In [5]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [6]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

C:\ProgramData\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
C:\ProgramData\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [7]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

In [8]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [9]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言文学: 适用于回答汉语言文学问题']


In [10]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题


In [11]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [12]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

<

In [13]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [14]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指处于温度T的物体所发出的辐射能量，它的光谱分布与温度有关，符合普朗克定律。它是一种理想的辐射体，具有吸收所有进入它表面的光线的能力，因此看起来是黑色的。它是研究光谱学和热辐射的重要理论基础。'}


In [16]:
print(chain.invoke("大于40的第一个质数是多少，使得这个质数加一能被3整除？"))



> Entering new MultiPromptChain chain...
数学: {'input': '第一个质数是多少'}
> Finished chain.
{'input': '第一个质数是多少', 'text': '？\n\n首先，我们需要明确质数的定义。质数是指只能被1和它本身整除的自然数。\n因此，我们需要从最小的自然数开始，依次判断每个数是否满足这个条件。\n\n首先是2，它只能被1和2整除，因此是质数。\n\n接下来是3，同样只能被1和3整除，也是质数。\n\n然后是4，它可以被1、2和4整除，不符合质数的定义。\n\n依此类推，我们可以得到第一个质数是2。 '}


In [17]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是一种极其密集的天体，它的引力非常强大，甚至连光也无法逃离它的吸引力。它的形成是由于一个超大质量的恒星在死亡时，它的内部压力无法抵抗重力，导致它坍缩成一个非常小而密集的物体。由于它的引力非常强大，所以它能够吸引周围的物质，甚至连光也无法逃离它的吸引力，因此被称为“黑洞”。目前还没有直接观测到黑洞，但是科学家们可以通过观测它们周围的物质运动和辐射来间接证明它们的存在。'}


In [18]:
print(chain.invoke("植物光合作用是在哪里进行的？"))



> Entering new MultiPromptChain chain...
生物: {'input': '植物光合作用是在哪里进行的？'}
> Finished chain.
{'input': '植物光合作用是在哪里进行的？', 'text': '\n\n植物的光合作用主要发生在叶片的叶绿体中。叶绿体是植物的细胞器，其中含有叶绿素和其他光合色素，它们可以吸收光能，并将其转化为化学能，用于光合作用的进行。'}


In [19]:
print(chain.invoke("信息论是由谁提出的？"))



> Entering new MultiPromptChain chain...
物理: {'input': '信息论是由谁提出的？物理'}
> Finished chain.
{'input': '信息论是由谁提出的？物理', 'text': '学家克劳德·香农提出了信息论这一概念。他在1948年的论文《通信的数学理论》中首次提出了信息论的概念。信息论是研究信息的量、传输和处理的理论。它对于理解通信系统和数据处理系统都非常重要。'}


In [20]:
print(chain.invoke("计算机中为何要使用二进制？"))



> Entering new MultiPromptChain chain...
计算机: {'input': '为何要使用二进制？'}
> Finished chain.
{'input': '为何要使用二进制？', 'text': '\n\n二进制是计算机中最基本的数制系统，它将所有的数据和指令都转换为0和1的二进制数来进行处理。这种方式的优势在于计算机内部电路可以用简单的开关来表示0和1，从而大大降低了电路复杂度。此外，二进制还具有高效、可靠和易于计算的特点，使得计算机可以快速地进行逻辑运算和存储数据。因此，使用二进制可以使计算机更加高效、快速和可靠地进行运算，进而实现各种复杂的计算任务。'}


In [21]:
print(chain.invoke("'学而时习之，不亦说乎'中的说是什么意思？"))



> Entering new MultiPromptChain chain...
汉语言文学: {'input': "'学而时习之，不亦说乎'中的说是什么意思？"}
> Finished chain.
{'input': "'学而时习之，不亦说乎'中的说是什么意思？", 'text': '\n\n在这句话中，"说"的意思是"快乐、满意"。这句话的意思是，学习的过程中如果能够及时复习和巩固所学的知识，那么就能够感到快乐和满意。这也是孔子教育理念中的重要观点之一。'}
